# Installation des dépendances

In [ ]:
!pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install diffusers[torch] transformers accelerate xformers fastai gradio peft

### 📚 Importations
- `torch` : pour le traitement avec PyTorch et le contrôle du générateur aléatoire.
- `StableDiffusionXLPipeline` & `DPMSolverMultistepScheduler` : composants essentiels de `diffusers` pour générer des images avec SDXL.
- `UNet2DConditionModel` & `PeftModel` : pour charger et appliquer le modèle UNet fine-tuné avec LoRA.
- `gradio` : bibliothèque pour créer facilement une interface web interactive.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler
from peft import PeftModel
import gradio as gr

### ✏️ Fonction `optimize_prompt`
Cette fonction génère :
- un **prompt positif** optimisé selon le produit, la couleur et la description, adapté au format du fine-tuning ;
- un **prompt négatif** pour exclure les défauts courants (basse qualité, déformations...).

Elle permet d'améliorer la qualité du rendu généré par le modèle Stable Diffusion XL fine-tuné.

In [ ]:
def optimize_prompt(product_type, color, description):
    phrasing = {
        "T-shirt": {
            "modif": "illustration détaillée sur t-shirt",
            "style": "style réaliste, haute résolution, éclairage dramatique, rendu artistique, impression de qualité"
        },
        "Mug": {
            "modif": "illustration détaillée sur mug",
            "style": "style réaliste, haute résolution, éclairage doux, rendu artistique, impression de qualité"
        },
        "Casquette": {
            "modif": "illustration brodée sur casquette",
            "style": "style simple, contraste élevé, impression nette, rendu textile"
        },
        "Coussin": {
            "modif": "motif harmonieux sur coussin",
            "style": "style décoratif, haute résolution, textile de qualité"
        },
        "Tableau": {
            "modif": "impression fine art sur toile",
            "style": "style artistique, composition équilibrée, détails fins, galerie d'art, rendu haut de gamme"
        }
    }
    this = phrasing.get(product_type, phrasing["T-shirt"])
    support = f"{this['modif']} {color}"
    optimized_prompt = f"{description}, {support}, {this['style']}"
    negative_prompt = "déformé, basse qualité, pixelisé, flou, proportions incorrectes, anatomie incorrecte, texte illisible, signature, logo, watermark"
    return optimized_prompt, negative_prompt

### ⚙️ Chargement du modèle
- On utilise **Stable Diffusion XL base 1.0** avec précision `float16` pour accélérer l’inférence.
- Le UNet est chargé et adapté avec **LoRA** via `PeftModel` pour intégrer le fine-tuning local.
- Le scheduler est remplacé par **DPMSolverMultistep** pour des résultats plus rapides et précis.
- Le pipeline est transféré sur GPU (`cuda`) pour de meilleures performances.

In [ ]:
MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
LORA_PATH = "fine_tuned_sdxl_ecommerce/unet_lora"  # Chemin vers tes poids LoRA sauvegardés

# Charger le UNet de base
unet = UNet2DConditionModel.from_pretrained(
    MODEL_ID, subfolder="unet",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

# Appliquer le LoRA local
unet = PeftModel.from_pretrained(unet, LORA_PATH)

# Fusionner pour l'inférence
unet = unet.merge_and_unload()

# Charger le pipeline sans UNet, puis assigner
pipe = StableDiffusionXLPipeline.from_pretrained(
    MODEL_ID,
    unet=unet,  # Assign the modified UNet
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# Scheduler pour des résultats rapides/précis
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

print("Modèle SDXL fine-tuné chargé et prêt !")

### 🖼️ Fonction `generate_image`
Elle permet de :
- Construire un **prompt optimisé** et un **prompt négatif**.
- Générer une image de **768x768 px** avec un nombre d’étapes et un `guidance scale` personnalisable.
- Utiliser un `seed` pour reproductibilité si fourni.

In [ ]:
def generate_image(product_type, color, description, steps, guidance, seed):
    prompt, negative_prompt = optimize_prompt(product_type, color, description)
    try:
        if seed is not None and int(seed) != -1:
            generator = torch.Generator("cuda").manual_seed(int(seed))
        else:
            generator = None
    except Exception:
        generator = None
    result = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=int(steps),
        guidance_scale=float(guidance),
        width=768,
        height=768,
        generator=generator
    )
    image = result.images[0]
    return image, prompt, negative_prompt

### 🧰 Interface Gradio
- Interface responsive avec **Gradio Blocks**.
- Entrées : produit, couleur, description + options avancées (`steps`, `guidance`, `seed`).
- Sorties : image générée, prompt optimisé, prompt négatif.
- `on_generate` : exécute la génération avec les valeurs d'entrée.
- `clear_fields` : réinitialise tous les champs aux valeurs par défaut.
- `generate_btn.click(...)` : connecte le bouton à la fonction de génération.
- `clear_btn.click(...)` : connecte le bouton à la fonction de réinitialisation.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("""
        # 🎨 <span style='color:#8f6ed5'>Générateur d'Images de Produits Personnalisés</span>
        <span style="font-size:1.1em;">
        Saisissez le type de produit, une couleur et votre idée <br>
        Obtenez un rendu <b>photoréaliste</b>&nbsp;: idéal pour vos boutiques en ligne !
        </span>
    """)
    with gr.Row():
        with gr.Column():
            product_type = gr.Dropdown(["T-shirt", "Mug", "Casquette", "Coussin", "Tableau"], label="Type de produit", value="T-shirt")
            color = gr.Dropdown(["blanc", "noir", "bleu", "rouge", "vert"], label="Couleur", value="blanc")
            description = gr.Textbox(label="Description", placeholder="Votre description ici")
            with gr.Accordion("Paramètres avancés", open=False):
                steps = gr.Slider(20, 50, value=30, step=1, label="Étapes de diffusion")
                guidance = gr.Slider(5.0, 9.0, value=7.5, step=0.1, label="Guidance Scale")
                seed = gr.Number(value=-1, label="Seed (-1 pour aléatoire)")
            generate_btn = gr.Button("✨ Générer")
            clear_btn = gr.Button("🧹 Effacer")
        with gr.Column():
            output_image = gr.Image(label="Aperçu produit", type="pil")
            optimized_prompt = gr.Textbox(label="Prompt optimisé réel")
            negative_prompt = gr.Textbox(label="Prompt négatif")

    # Mover les appels .click() à l'intérieur du bloc `with gr.Blocks() as demo:`
    def on_generate(*vals):
        return generate_image(*vals)

    def clear_fields():
        # Restaure les valeurs par défaut
        return ("T-shirt", "blanc", "", 30, 7.5, -1, None, "", "")

    generate_btn.click(
        on_generate,
        inputs=[product_type, color, description, steps, guidance, seed],
        outputs=[output_image, optimized_prompt, negative_prompt]
    )
    clear_btn.click(
        clear_fields,
        None,
        [product_type, color, description, steps, guidance, seed, output_image, optimized_prompt, negative_prompt]
    )

### 🚀 Lancement de l'application Gradio
- L’interface est lancée avec `share=True` pour obtenir un **lien public** accessible depuis n’importe quel navigateur.

In [ ]:
demo.launch(share=True)